In [83]:
import sys
sys.path.append("../src")
import pickle as pkl
from pathlib import Path

import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer, util

from gen.util import read_data

# Init

In [2]:
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

sf_doc_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/thesis/errors/scifact/document")
sf_sent_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/thesis/errors/scifact/sentence")

sf_pipe_doc_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/thesis/errors/scifactpipeline/document")
sf_pipe_sent_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/thesis/errors/scifactpipeline/sentence")

In [84]:
result_p = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/thesis/results/metrics")

for res in result_p.iterdir():
    print(res.stem)
    if res.suffix == ".pkl":
        with res.open("rb") as fn:
            globals()[res.stem] = pkl.load(fn)
    elif res.suffix == ".jsonl":
        globals()[res.stem] = read_data(res)

scifact_pipeline_scifact_evaluator
concatenate_evidences_metrics
sent_macro_verdict_meanproba_metrics
sent_micro_verdict_metrics
sent_macro_verdict_majority_metrics


In [3]:
def doc_reader(data_dir):
    res = []
    for p in data_dir.iterdir():
        tok = p.stem.split("_")
        df = pd.DataFrame(read_data(p))
        df["aggregation"] = "document"
        df["agreement"] = tok[2]
        df["dataset"] = None if tok[2] == "agree" else tok[3]
        
        res.append(df)
    
    return pd.concat(res, axis=0).set_index("id")

def sent_reader(data_dir):
    res = []
    for p in data_dir.iterdir():
        tok = p.stem.split("_")
        df = pd.DataFrame(read_data(p))
        df["aggregation"] = tok[0]
        df["agreement"] = tok[2]
        df["dataset"] = None if tok[2] == "agree" else tok[3]
        if "id" not in df:
            df["id"] = df["index"]
            df = df.drop(columns=["index"])
        
        res.append(df)
    
    return pd.concat(res, axis=0).set_index("id")

def simple_cosine(claim, evidences):
    claim_embedding = sbert_model.encode([claim], convert_to_tensor=True)
    evidence_embedding = sbert_model.encode(evidences, convert_to_tensor=True)    
    
    return util.cos_sim(claim_embedding, evidence_embedding)

In [147]:
simple_cosine("acute paraquat poisoning", ["acute kidney injury", "severe PQ poisoning"])

tensor([[0.3580, 0.5156]], device='cuda:0')

# Oracle IR

## Document

In [122]:
sf_docs = doc_reader(sf_doc_p)
sf_docs.shape

(395, 8)

In [126]:
sf_docs.query("agreement == 'disagree' and dataset == 'alltrain'").sort_values("cosine_score", ascending=False)

,claim,evidence,labels,cosine_score,model_agreement,aggregation,agreement,dataset
id,,,,,,,,
scifact|123,Antiretroviral therapy increases rates of tube...,Antiretroviral therapy is strongly associated ...,2,0.8927,"[[0, 8], [1, 1]]",document,disagree,alltrain
scifact|566,"In S. cerevisiae, the absence of RNA surveilla...",We show that many mRNAs encoding iron uptake o...,2,0.8331,"[[0, 7], [1, 2]]",document,disagree,alltrain
scifact|1050,Risedronate increases risk of vertebral and no...,"Treatment with 5 mg/d of risedronate, compared...",2,0.7995,"[[0, 7], [1, 2]]",document,disagree,alltrain
scifact|1360,Varenicline monotherapy is more effective afte...,"At 26 weeks, 36.6% of the combination therapy ...",2,0.7793,"[[0, 8], [1, 1]]",document,disagree,alltrain
scifact|1288,There is a positive correlation between hip fr...,RESULTS Use of statins in either the prior 180...,2,0.7544,"[[0, 5], [1, 4]]",document,disagree,alltrain
scifact|1359,Varenicline monotherapy is more effective afte...,"RESULTS At 12 weeks, 53.0% of the combination ...",2,0.7499,"[[0, 8], [1, 1]]",document,disagree,alltrain
scifact|1170,The NF-κB essential modulator gene induces int...,Inhibition of receptor-interacting protein kin...,2,0.7447,"[[0, 8], [1, 1]]",document,disagree,alltrain
scifact|1270,The risk of male prisoners harming themselves ...,"FINDINGS 139,195 self-harm incidents were reco...",2,0.7237,"[[0, 8], [1, 1]]",document,disagree,alltrain
scifact|69,Activation of PPM1D enhances p53 function.,PPM1D mutations were mutually exclusive with T...,2,0.7141,"[[0, 8], [1, 1]]",document,disagree,alltrain


In [139]:
sf_docs.loc["scifact|246"]

,claim,evidence,labels,cosine_score,model_agreement,aggregation,agreement,dataset
id,,,,,,,,
scifact|246,Charcoal shows no benefit for acute paraquat p...,Activated charcoal hemoperfusion had little ef...,0,0.5249,"[[2, 3]]",document,disagree,fever-climatefeverpure
scifact|246,Charcoal shows no benefit for acute paraquat p...,Activated charcoal hemoperfusion had little ef...,0,0.5249,"[[2, 3]]",document,disagree,fever


## Sentence

In [119]:
sf_sents = sent_reader(sf_sent_p)
sf_sents.shape

(3741, 11)

In [134]:
sf_sents.loc[
    ((sf_sents["dataset"] == "alltrain") & (sf_sents["agreement"] == "disagree")) & 
    (sf_sents["labels"] == 0) 
].sort_values("cosine_score", ascending=False)

,claim,evidence,cosine_score,labels,predicted_proba,mean_proba,aggregation,agreement,dataset,predicted_label,model_agreement
id,,,,,,,,,,,
scifact|593,Incidence of heart failure decreased by 10% in...,[RESULTS The incidence of heart failure was hi...,[0.6757000088691711],0,"{'climatefever': {'xlnet': [[0.3242, 0.4457, 0...","{'climatefever': {'xlnet': [0.3242, 0.4457, 0....",meanproba,disagree,alltrain,NaN,NaN
scifact|593,Incidence of heart failure decreased by 10% in...,[RESULTS The incidence of heart failure was hi...,[0.6757000088691711],0,NaN,NaN,majority,disagree,alltrain,"{'climatefever': {'xlnet': [1], 'bert': [2]}, ...",NaN
scifact|184,Bone marrow cells don't contribute to adult ma...,[We also found that after bone marrow transpla...,[0.5990999937057495],0,"{'climatefever': {'xlnet': [[0.0735, 0.8398, 0...","{'climatefever': {'xlnet': [0.0735, 0.8398, 0....",meanproba,disagree,alltrain,NaN,NaN
scifact|184,Bone marrow cells don't contribute to adult ma...,[We also found that after bone marrow transpla...,[0.5990999937057495],0,NaN,NaN,majority,disagree,alltrain,"{'climatefever': {'xlnet': [1], 'bert': [2]}, ...",NaN
scifact|246,Charcoal shows no benefit for acute paraquat p...,[Activated charcoal hemoperfusion had little e...,[0.5249000191688538],0,"{'climatefever': {'xlnet': [[0.0475, 0.3774, 0...","{'climatefever': {'xlnet': [0.0475, 0.3774, 0....",meanproba,disagree,alltrain,NaN,NaN
scifact|246,Charcoal shows no benefit for acute paraquat p...,[Activated charcoal hemoperfusion had little e...,[0.5249000191688538],0,NaN,NaN,majority,disagree,alltrain,"{'climatefever': {'xlnet': [2], 'bert': [2]}, ...",NaN
scifact|12,40mg/day dosage of folic acid and 2mg/day dosa...,[RESULTS Mean baseline homocysteine level was ...,"[0.507099986076355, 0.7872999906539917]",0,"{'climatefever': {'xlnet': [[0.0137, 0.8813, 0...","{'climatefever': {'xlnet': [0.0296, 0.7806, 0....",meanproba,disagree,alltrain,NaN,NaN
scifact|12,40mg/day dosage of folic acid and 2mg/day dosa...,[RESULTS Mean baseline homocysteine level was ...,"[0.507099986076355, 0.7872999906539917]",0,NaN,NaN,majority,disagree,alltrain,"{'climatefever': {'xlnet': [1, 1], 'bert': [1,...",NaN
scifact|39,A diminished ovarian reserve does not solely i...,"[After adjusting for age, body mass index, rac...","[0.46309998631477356, 0.337799996137619, 0.332...",0,"{'climatefever': {'xlnet': [[0.0231, 0.7807, 0...","{'climatefever': {'xlnet': [0.0233, 0.7325, 0....",meanproba,disagree,alltrain,NaN,NaN


In [140]:
sf_sents.loc["scifact|718"]

,claim,evidence,cosine_score,labels,predicted_proba,mean_proba,aggregation,agreement,dataset,predicted_label,model_agreement
id,,,,,,,,,,,
scifact|718,Low nucleosome occupancy correlates with low m...,[Clustered methylation occurs at unprecedented...,[0.6699000000953674],2,"{'bert': [[0.4849, 0.4824, 0.0326]], 'xlnet': ...","{'bert': [0.4849, 0.4824, 0.0326], 'xlnet': [0...",meanproba,disagree,climatefever,NaN,NaN
scifact|718,Low nucleosome occupancy correlates with low m...,[Clustered methylation occurs at unprecedented...,[0.6699000000953674],2,"{'bert': [[0.9545, 0.0061, 0.0393]], 'xlnet': ...","{'bert': [0.9545, 0.0061, 0.0393], 'xlnet': [0...",meanproba,disagree,climatefeverpure,NaN,NaN
scifact|718,Low nucleosome occupancy correlates with low m...,[Clustered methylation occurs at unprecedented...,[0.6699000000953674],2,"{'bert': [[0.053, 0.9179, 0.0291]], 'xlnet': [...","{'bert': [0.053, 0.9179, 0.0291], 'xlnet': [0....",meanproba,disagree,fever-climatefever,NaN,NaN
scifact|718,Low nucleosome occupancy correlates with low m...,[Clustered methylation occurs at unprecedented...,[0.6699000000953674],2,NaN,NaN,majority,disagree,climatefeverpure,"{'bert': [0], 'xlnet': [0]}",NaN
scifact|718,Low nucleosome occupancy correlates with low m...,[Clustered methylation occurs at unprecedented...,[0.6699000000953674],2,NaN,NaN,majority,disagree,climatefever,"{'bert': [0], 'xlnet': [0]}",NaN
scifact|718,Low nucleosome occupancy correlates with low m...,[Clustered methylation occurs at unprecedented...,[0.6699000000953674],2,NaN,NaN,majority,disagree,fever-climatefeverpure,"{'bert': [1], 'xlnet': [0]}",NaN
scifact|718,Low nucleosome occupancy correlates with low m...,[Clustered methylation occurs at unprecedented...,[0.6699000000953674],2,NaN,NaN,majority,disagree,fever-climatefever,"{'bert': [1], 'xlnet': [0]}",NaN
scifact|718,Low nucleosome occupancy correlates with low m...,[Clustered methylation occurs at unprecedented...,[0.6699000000953674],2,"{'bert': [[0.0, 0.9998, 0.0002]], 'xlnet': [[0...","{'bert': [0.0, 0.9998, 0.0002], 'xlnet': [0.99...",meanproba,disagree,fever-climatefeverpure,NaN,NaN


# Pipeline

## Instances with completely irrelevant evidence

In [40]:
import constants
from scoring.scorer import FEVERScorer

sf_pipe_data = read_data(Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/bert-data-sent-evidence/scifactpipeline.all.test.n6.s5.jsonl"))
sf_actual_data = read_data(Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/scifact-nei-sampled/all.n5.nei.jsonl"))

for i, doc in enumerate(sf_actual_data):
    if doc["label"] == constants.LOOKUP["label"]["nei"]:
        sf_actual_data[i]["evidence"] = sf_actual_data[i]["evidence"][:2]
sf_pipe_data = (
    pd.DataFrame(sf_pipe_data)
    .groupby("claim_id", as_index=False, sort=False)
    .agg({"predicted_evidence": list})
)

In [78]:
sf_scorer = FEVERScorer(sf_actual_data, sf_pipe_data.to_dict("records"), oracle_rte=True, oracle_ir=False)
sf_match = pd.DataFrame(sf_scorer.match_matrix)
sf_match = sf_match.assign(
    id="scifact|" + sf_match["claim_id"].astype(str),
    absent_evidence=sf_match["n_total_evidences"] - sf_match["full_hit"],
    retrieve_evidence_pct=round(sf_match["full_hit"]/sf_match["n_total_evidences"], 4)
)
sf_match = sf_match.assign(
    retrieve_evidence_bin=pd.cut(sf_match["retrieve_evidence_pct"], bins=[-1,0,.99,1])
)
sf_match = sf_match.set_index("id").drop(columns=["claim_id"])
sf_match.shape

(1109, 11)

In [81]:
sf_match.query("retrieve_evidence_pct == 0")

,claim_label,n_predicted_evidences,n_total_evidences,full_hit,evidence_sent_miss,irrelevant,evidence_page,predicted_page,absent_evidence,retrieve_evidence_pct,retrieve_evidence_bin
id,,,,,,,,,,,
scifact|79,SUPPORTS,5,2,0.0,1,"{4345315, 4407455}",{5099266},"{4345315, 4407455, 5099266}",2.0,0.0,"(-1.0, 0.0]"
scifact|88,SUPPORTS,5,1,0.0,0,"{10015292, 16627684, 13116880, 12827098}",{7521113},"{10015292, 16627684, 13116880, 12827098}",1.0,0.0,"(-1.0, 0.0]"
scifact|93,SUPPORTS,5,2,0.0,0,"{5386514, 6000423, 4303939, 4399311}",{2692522},"{5386514, 6000423, 4303939, 4399311}",2.0,0.0,"(-1.0, 0.0]"
scifact|111,SUPPORTS,5,2,0.0,1,"{7581911, 4784069, 13910150, 4416964}",{13513790},"{13513790, 13910150, 4784069, 7581911, 4416964}",2.0,0.0,"(-1.0, 0.0]"
scifact|121,SUPPORTS,5,1,0.0,0,"{12801438, 1234098, 43165768, 36497180}",{31460499},"{12801438, 1234098, 43165768, 36497180}",1.0,0.0,"(-1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...
scifact|1319,SUPPORTS,5,1,0.0,2,"{4388470, 14192687}",{16284655},"{4388470, 14192687, 16284655}",1.0,0.0,"(-1.0, 0.0]"
scifact|1320,REFUTES,5,1,0.0,2,"{2437807, 3360421}",{16284655},"{2437807, 16284655, 3360421}",1.0,0.0,"(-1.0, 0.0]"
scifact|1368,SUPPORTS,5,4,0.0,0,"{13312471, 8856690, 23267371, 275294}",{2425364},"{23267371, 13312471, 8856690, 275294}",4.0,0.0,"(-1.0, 0.0]"


In [79]:
sf_match.reset_index().pivot_table(index="retrieve_evidence_bin", columns="claim_label", values="id", aggfunc="count")

claim_label,NOT ENOUGH INFO,REFUTES,SUPPORTS
retrieve_evidence_bin,,,
"(-1.0, 0.0]",0,58,103
"(0.0, 0.99]",1,77,115
"(0.99, 1.0]",415,102,238


In [105]:
best_doc = pd.DataFrame(concatenate_evidences_metrics["scifactpipeline"]["all"][7].predictions).rename(columns={"predicted_label":"doc"})
best_maj = pd.DataFrame(sent_macro_verdict_majority_metrics["scifactpipeline"]["all"][0].predictions).rename(columns={"predicted_label":"maj"})
best_mean = pd.DataFrame(sent_macro_verdict_meanproba_metrics["scifactpipeline"]["all"][0].predictions).rename(columns={"predicted_label":"meanproba"})

df_retr_influence = pd.concat([
    sf_match[["claim_label", "retrieve_evidence_bin"]].reset_index(),
    best_doc["doc"], best_maj["maj"], best_mean["meanproba"]
], axis=1)
df_retr_influence = df_retr_influence.assign(
    doc_true=df_retr_influence["claim_label"]==df_retr_influence["doc"],
    maj_true=df_retr_influence["claim_label"]==df_retr_influence["maj"],
    meanproba_true=df_retr_influence["claim_label"]==df_retr_influence["meanproba"]
)

In [112]:
(
    df_retr_influence
    .query("claim_label != 'NOT ENOUGH INFO'")
    .pivot_table(index="retrieve_evidence_bin", columns="claim_label", values="meanproba_true", aggfunc="sum")
)

claim_label,REFUTES,SUPPORTS
retrieve_evidence_bin,,
"(-1.0, 0.0]",24,64
"(0.0, 0.99]",26,80
"(0.99, 1.0]",37,143


## Document

In [4]:
sf_docs_pipe = doc_reader(sf_pipe_doc_p)
sf_docs_pipe.shape

(859, 9)

In [12]:
sf_docs_pipe.loc[
    (sf_docs_pipe["agreement"] == "agree")
].sort_values("cosine_score", ascending=False)

,claim,evidence,predicted_evidence,cosine_score,labels,model_agreement,aggregation,agreement,dataset
id,,,,,,,,,
scifact|621,Individuals with Alzheimers who participate in...,CONCLUSIONS In this study of adults with subje...,"[[1642727, 13], [17691617, 10], [17691617, 9],...",0.7796,0,"[[1, 9]]",document,agree,None
scifact|495,Having a main partner improves HIV outcomes.,CONCLUSIONS A stable partnership is associated...,"[[17077004, 9], [17077004, 0], [5835149, 4], [...",0.7225,0,"[[1, 9]]",document,agree,None
scifact|1327,Tuberculosis incidence occurs at higher rates ...,RESULTS Univariate analysis showed significant...,"[[24241932, 5], [24241932, 1], [24241932, 7], ...",0.7093,2,"[[1, 9]]",document,agree,None
scifact|498,Having a main partner worsens HIV outcomes.,MAIN OUTCOME MEASURES Prevalence of HCV antibo...,"[[5835149, 4], [17077004, 0], [17077004, 9], [...",0.6894,2,"[[1, 9]]",document,agree,None
scifact|115,Anthrax spores can be disposed of easily after...,More than 80% of the B anthracis particles col...,"[[33872649, 6], [33872649, 4], [33872649, 1], ...",0.6575,2,"[[1, 9]]",document,agree,None
scifact|396,Exercise increases cancer mortality rates amon...,"CONCLUSIONS In this first study, to our knowle...","[[1456068, 11], [24918110, 9], [1456068, 0], [...",0.6530,2,"[[1, 9]]",document,agree,None
scifact|114,Anthrax spores are very difficult to dispose o...,RESULTS Viable B anthracis spores reaerosolize...,"[[33872649, 4], [33872649, 6], [33872649, 1], ...",0.6422,0,"[[1, 9]]",document,agree,None


## Sentence

In [6]:
sf_sent_pipe = sent_reader(sf_pipe_sent_p)
sf_sent_pipe.shape

(3789, 11)

In [113]:
sf_sent_pipe.loc[["scifact|290"]]

,claim,evidence,cosine_score,labels,predicted_proba,mean_proba,aggregation,agreement,dataset,predicted_label,model_agreement
id,,,,,,,,,,,
scifact|290,Cost effectiveness evaluations based on cRCT d...,[The objective of this study was to evaluate t...,"[0.7166000008583069, 0.5605000257492065, 0.643...",2,"{'climatefever': {'xlnet': [[0.6442, 0.3342, 0...","{'climatefever': {'xlnet': [0.513, 0.4572, 0.0...",meanproba,disagree,alltrain,NaN,NaN
scifact|290,Cost effectiveness evaluations based on cRCT d...,[The objective of this study was to evaluate t...,"[0.7166000008583069, 0.5605000257492065, 0.643...",2,"{'bert': [[0.697, 0.2883, 0.0147], [0.3488, 0....","{'bert': [0.4509, 0.5312, 0.018], 'xlnet': [0....",meanproba,disagree,climatefever,NaN,NaN
scifact|290,Cost effectiveness evaluations based on cRCT d...,[The objective of this study was to evaluate t...,"[0.7166000008583069, 0.5605000257492065, 0.643...",2,"{'bert': [[0.9656, 0.0052, 0.0292], [0.964, 0....","{'bert': [0.966, 0.0087, 0.0254], 'xlnet': [0....",meanproba,disagree,climatefeverpure,NaN,NaN
scifact|290,Cost effectiveness evaluations based on cRCT d...,[The objective of this study was to evaluate t...,"[0.7166000008583069, 0.5605000257492065, 0.643...",2,"{'bert': [[0.0618, 0.9283, 0.0099], [0.0058, 0...","{'bert': [0.0345, 0.9561, 0.0093], 'xlnet': [0...",meanproba,disagree,fever-climatefever,NaN,NaN
scifact|290,Cost effectiveness evaluations based on cRCT d...,[The objective of this study was to evaluate t...,"[0.7166000008583069, 0.5605000257492065, 0.643...",2,NaN,NaN,majority,disagree,climatefeverpure,"{'bert': [0, 0, 0, 0, 0], 'xlnet': [0, 2, 0, 1...",NaN
scifact|290,Cost effectiveness evaluations based on cRCT d...,[The objective of this study was to evaluate t...,"[0.7166000008583069, 0.5605000257492065, 0.643...",2,NaN,NaN,majority,disagree,climatefever,"{'bert': [0, 1, 0, 1, 1], 'xlnet': [0, 0, 0, 1...",NaN
scifact|290,Cost effectiveness evaluations based on cRCT d...,[The objective of this study was to evaluate t...,"[0.7166000008583069, 0.5605000257492065, 0.643...",2,NaN,NaN,majority,disagree,fever-climatefeverpure,"{'bert': [1, 1, 1, 1, 1], 'xlnet': [0, 2, 0, 0...",NaN
scifact|290,Cost effectiveness evaluations based on cRCT d...,[The objective of this study was to evaluate t...,"[0.7166000008583069, 0.5605000257492065, 0.643...",2,"{'bert': [[0.9701, 0.0, 0.0299], [0.0551, 0.00...","{'bert': [0.7587, 0.0004, 0.2409], 'xlnet': [0...",meanproba,disagree,fever,NaN,NaN
scifact|290,Cost effectiveness evaluations based on cRCT d...,[The objective of this study was to evaluate t...,"[0.7166000008583069, 0.5605000257492065, 0.643...",2,NaN,NaN,majority,disagree,alltrain,"{'climatefever': {'xlnet': [0, 0, 0, 1, 0], 'b...",NaN
